In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns',None)
pd.options.display.max_rows = 30

## 불러오기

In [ ]:
## Snowflake Cloud 접속 정보
snowflake_account = 'ls58031.ap-northeast-2.privatelink'
snowflake_user='21600594'
snowflake_password='test123!A'
snowflake_database='HKT_DW'
snowflake_warehouse='SMALL_WH'
snowflake_schema = ''

## Oracle DB 접속 정보
oracle_user = 'hqgmes'
oracle_password = 'hqgmes'
oracle_dsn = '202.31.25.83:1521/DKPPODA.kppodad'

# Oracle Clinet 설치 결로
oracle_dir = r"C:\Users\hanta\Downloads\instantclient_21_10"
print('done')

In [ ]:
# !pip install "snowflake-connector-python[pandas]"
# !pip install oracledb
import snowflake.connector
import oracledb
oracledb.init_oracle_client(lib_dir=oracle_dir)

def db_table(quary, db_type):
    # snowflake 데이터 불러오기
    if db_type == 'sf' :
        con = snowflake.connector.connect(
            account = snowflake_account,
            user = snowflake_user,
            password = snowflake_password,
            database = snowflake_database,
            warehouse = snowflake_warehouse
            )
        cur = con.cursor()
        cur.execute(quary)
        result_df = cur.fetch_pandas_all()
        con.close
        return result_df
    
    # Oracle DB 데이터 불러오기
    elif db_type == 'oda':
        con = oracledb.connect(
            user = oracle_user, 
            password = oracle_password, 
            dsn = oracle_dsn
            )
        cur = con.cursor()
        cur.execute(quary)
        table_cols = [cols[0] for cols in cur.description]
        result_df = pd.DataFrame(cur.fetchall(), columns=table_cols)
        con.close
        return result_df 
    else:
        print('oda 또는 sf을 변수로 입력하세요')

In [ ]:
from datetime import datetime, timedelta

# 정수 형식의 날짜 (예: 20230101)
date_int = 20230101

# 정수를 날짜형으로
date_obj = datetime.strptime( str(date_int), '%Y%m%d')

# 15일 전의 날짜 계산
date_15_days_ago = date_obj - timedelta(days=15)

# 결과 출력 (다시 정수로 변환할 수 있음)
pre_ref_date1 = int(date_15_days_ago.strftime('%Y%m%d'))



In [ ]:
ref_date1 = 20230501 # 2023년 01월
ref_date2 = 20230630 # 2023년 12월




# 15일 전/후의 날짜 계산
date_obj = datetime.strptime( str(ref_date1), '%Y%m%d')
date_15_days_ago = date_obj - timedelta(days=15)

date_obj2 = datetime.strptime( str(ref_date2), '%Y%m%d')
date_15_days_later = date_obj2 + timedelta(days=15)

pre_ref_date1 = int(date_15_days_ago.strftime('%Y%m%d'))
after_ref_date2 = int(date_15_days_later.strftime('%Y%m%d'))




# MES 기준 : LWD307 - 정련 운전 실적 / RUNSUMMARY
LQLTTR164 = f'''   
    SELECT 
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,WRK_ORD_NO
        ,ORD_SEQ
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,MC_RUN_MODE
        ,PRDT_DATE
        ,MAX(BTCH_NO) AS BTCH_NO
        ,MIN(WRK_SRT_DTM) AS WRK_SRT_DTM
        ,MAX(WRK_END_DTM) AS WRK_END_DTM
    FROM DPPMES.LQLTTR164
    WHERE 1=1
        AND LOT_NO BETWEEN '{ref_date1}' AND '{ref_date2}'
        -- AND LOT_NO = 202301040002
        -- AND SPEC_CD = 'C1C49S019'
        -- AND MC_CD = 'A1002'
        -- ORDER BY 1,2,3,4,5,6
    GROUP BY   
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,WRK_ORD_NO
        ,ORD_SEQ
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,MC_RUN_MODE
        ,PRDT_DATE
'''

# MES 기준 : LWD333 - 컴파운드 생산별 투입 원부재, 고무
LQLTTR147 = f'''
    SELECT 
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,BTCH_NO
        ,RMTL_TP_CD 
        ,RMTL_BIN_NO 
        ,RMTL_IPT_DGE
        ,RMTL_CD
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,PRDT_DTM
        ,UDT_DTM
        ,MRM_WGT
    FROM DPPMES.LQLTTR147
    WHERE 1=1
        AND LOT_NO BETWEEN '{ref_date1}' AND '{ref_date2}'
        -- AND LOT_NO = 202304160001
        -- AND SPEC_CD = 'C1U36T020'
        -- AND MC_CD = 'A1002'
'''

# MES 기준 : LWD841 - 원부재 투입 실적
LQLTTR151 = f'''
    SELECT 
        PLT_CD
        ,SCAN_DTM
        ,MC_CD
        -- ,SCAN_RMTL_CD
        ,PRDT_DATE
        ,RMTL_TP_CD
        ,RMTL_CD
        ,LOT_NO
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,WRK_FG
        ,WRK_FG_NM
        ,SCAN_BARCD_NO
        ,WHS_LOT_NO
        ,CMP_CD
        ,CMP_NM    
    FROM DPPMES.LQLTTR151
    WHERE SCAN_DTM BETWEEN '{pre_ref_date1}' AND '{ref_date2}'
'''

# ODA TRXX_TRACE 데이터
TRXX_TRACE = f'''
    SELECT *
    FROM BI_XV_TRXX_TRACE_SETA
    WHERE PLN_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE060 : RPA 검사 데이터 
LQLTTR268 = f'''
    SELECT * 
    FROM DPPMES.LQLTTR268
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE431 : BATCH 검사 데이터
LQLTTR171 = f'''
    SELECT *
    FROM DPPMES.LQLTTR171
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE461 : 고무물성(정밀) 검사 데이터
LQLTTR177 = f'''
    SELECT *
    FROM DPPMES.LQLTTR177
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''
    

In [ ]:

# ODA TRXX_TRACE 데이터
TRXX_TRACE = f'''
    SELECT *
    FROM BI_XV_TRXX_TRACE_SETA
    WHERE PLN_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
    AND PLT_CD = 'DP'
'''

In [ ]:
df_lwd333_lqlttr147 = db_table(LQLTTR147,'sf')
df_lwd307_lqlttr164 = db_table(LQLTTR164,'sf')
df_lwd841_lqlttr151 = db_table(LQLTTR151,'sf')
df_trxx_trace = db_table(TRXX_TRACE,'oda')

df_lqe060_lqlttr268 = db_table(LQLTTR268,'sf')
df_lqe431_lqlttr171 = db_table(LQLTTR171,'sf')
df_lqe461_lqlttr177 = db_table(LQLTTR177,'sf')

print (
    f'''
    lwd333 : df size = {df_lwd333_lqlttr147.shape} / 컴파운드 생산별 투입 원부재, 고무
    lwd307 : df size = {df_lwd307_lqlttr164.shape} / 정련 운전 실적 / RUNSUMMARY
    lwd841 : df size = {df_lwd841_lqlttr151.shape} / 원부재 투입 실적
    trace : df size = {df_trxx_trace.shape} /trace 데이터

    lqe060 : {df_lqe060_lqlttr268.shape} /RPA 검사 데이터
    lqe431 : {df_lqe431_lqlttr171.shape} /BATCH 데이터
    lqe461 : {df_lqe461_lqlttr177.shape} /고무 물성 정밀검사 데이터
'''
)

In [ ]:
# 스캔날짜 int로 바꾸기
df_lwd841_lqlttr151['SCAN_DTM'] = pd.to_numeric(df_lwd841_lqlttr151['SCAN_DTM'], errors='coerce', downcast='integer')
df_lwd841_lqlttr151.head()

## GLP 출고 데이터 → 원료 투입 데이터 (841) 꼴로 강제 변형 후 합치기

In [ ]:
dfrelease_path = r"C:\Users\hanta\OneDrive - HKNC\Arena Messenger Download\6. 출고이력 RAW_WM_P_L2141_2023-09-20 (2).csv"

In [ ]:
dfrelease = pd.read_csv(dfrelease_path)
dfrelease.rename (columns = {'생성일자':'CREDAT' , 
                             '생성일시':'CREDATIM', 
                             'Material':'SKUKEY', 
                             'LOT':'LOTA02', 
                             "업체":"LOTA01", 
                             "W/H":"SWERKS"} , inplace = True)
dfrelease.head()

# 생성일자 = CREDAT
# 생성일시 = CREDAT + CRETIM
# Material = SKUKEY
# LOT = LOTA02
# 업체 = LOTA01
# W/H = SWERKS

In [ ]:
# LOT 정보가 없는 원재료바코드투입실적 삭제

print ( 'LOT 정보 없는 행 개수 :', len (df_lwd841_lqlttr151[df_lwd841_lqlttr151['WHS_LOT_NO']==' ']) )

df_lwd841_lqlttr151 = df_lwd841_lqlttr151.drop ( df_lwd841_lqlttr151[df_lwd841_lqlttr151['WHS_LOT_NO']==' '].index)

print ( 'LOT 정보 있는 행 개수 :', len (df_lwd841_lqlttr151) )

In [ ]:
# 스캔날짜 int로 바꾸기
dfrelease['CREDATIM'] = pd.to_numeric(dfrelease['CREDATIM'], errors='coerce', downcast='integer')

In [ ]:
# 원료 정련 투입 이력 →  강제로 LWD841꼴로 만들기 (설비란은 비워둠)
df_lwd841_lqlttr151_release = df_lwd841_lqlttr151.copy()
df_lwd841_lqlttr151_release = df_lwd841_lqlttr151_release.iloc[0:0]  # 내용 다 비우기


df_lwd841_lqlttr151_release['SCAN_DTM'] =  dfrelease['CREDATIM']
df_lwd841_lqlttr151_release['PLT_CD'] =  'DP'
df_lwd841_lqlttr151_release['CMP_NM'] = dfrelease['업체명']

df_lwd841_lqlttr151_release['CMP_CD']= dfrelease['LOTA01']
df_lwd841_lqlttr151_release['RMTL_CD'] = dfrelease['SKUKEY']
df_lwd841_lqlttr151_release['WHS_LOT_NO'] = dfrelease['LOTA02']

# dflwd841_release['GR Date'] = dfrelease['Create Date']
# dflwd841_release['작업자'] = dfrelease['Keynum']

df_lwd841_lqlttr151_release.head()


# 얘네는 정말 정련 출고 데이터라, 어느 설비 (믹싱기)로 들어가는 지 정보가 없음
# 그래서 날짜 기준으로 넣을 예정 (같은 Material의 다른 LOT 출고 전까지, 이전 LOT 를 컴파운드에 넣는다는 가정)
# 이미 매칭된 애들 (바코드 정보 남아있는 것) 제외하고 컴파운드에 매칭할 것


In [ ]:
df_lwd841_lqlttr151_release[df_lwd841_lqlttr151_release['RMTL_CD'].str[:3]=='AAC']

In [ ]:
# GLP 출고 + MES LWD841 합치기

lwd841tot = pd.concat([df_lwd841_lqlttr151_release, df_lwd841_lqlttr151], axis=0)
lwd841tot.sort_values(by='SCAN_DTM')
print(f"""
    lwd841_lqlttr151 : {df_lwd841_lqlttr151.shape}
    dfrelease : {df_lwd841_lqlttr151_release.shape}
    joined(lwd841tot) : {lwd841tot.shape}
    """)
print("="*100)

# 몇개 열만 추출
selected_columns = ['PLT_CD', 'SCAN_DTM', 'MC_CD', 'PRDT_DATE', 'RMTL_CD','WHS_LOT_NO', 'CMP_CD', 'CMP_NM' ]
df_selected_841 = lwd841tot[selected_columns]

df_selected_841

## 컴파운드 생산에 들어간 전체 원료-LOT 연결 (LWD333+LWD841)

In [ ]:
df_lwd333_lqlttr147

In [ ]:
df_lwd333_lqlttr147['UDT_DTM'].str[8:]

In [ ]:
test = df_lwd333_lqlttr147[df_lwd333_lqlttr147['PRDT_DTM']!=df_lwd333_lqlttr147['UDT_DTM'].str[:8]]
test[['PRDT_DTM','UDT_DTM']]

In [ ]:
# 한 믹싱런에서는 같은 원료 들어갔다고 가정하고, PLT, MC, SPEC일치하면 RMTL 유니크하게 남겨두기

df_lwd333_lqlttr147_uniqueRM = df_lwd333_lqlttr147.drop_duplicates(subset=['PLT_CD', 'LOT_NO','MC_CD', 'SPEC_CD', 'RMTL_CD'], keep='first')

print(  "생산때 들어간 원재료 + 이전고무 총 개수: {0}".format(len(df_lwd333_lqlttr147_uniqueRM))  )

df_lwd333_lqlttr147_uniqueRM


#### 설비가 비어있는 것 (정련출고만 있는 경우) : 원부재, 공장 보고 합치기 (설비 안 봄)

In [ ]:
df_lwd333_lqlttr147_uniqueRM

In [ ]:
df_selected_841

In [ ]:
# 설비가 비어있는 것 (정련출고 only) 합치기
c1 = df_selected_841['MC_CD'].isna()
merged_run_release = pd.merge(df_lwd333_lqlttr147_uniqueRM, df_selected_841[c1], on=['RMTL_CD', "PLT_CD"],how='left')
print(merged_run_release.shape, df_lwd333_lqlttr147_uniqueRM.shape, df_selected_841[c1].shape)
print("-"*100)

merged_run_release['UDT_DTM']= pd.to_numeric(merged_run_release['UDT_DTM'], errors='coerce', downcast='integer')
merged_run_release['SCAN_DTM']= pd.to_numeric(merged_run_release['SCAN_DTM'], errors='coerce', downcast='integer')



# 투입 일자가 정련 시작보다 직전인 것 중에 가장 큰 값만 남기기
filtered_df = merged_run_release[merged_run_release['SCAN_DTM'] <= merged_run_release['UDT_DTM']]
max_d_row = filtered_df.groupby(['MC_CD_x', 'LOT_NO', 'SPEC_CD', 'RMTL_CD'])['SCAN_DTM'].idxmax()
filtered_merged_run_release = filtered_df.loc[max_d_row]


print(  "정련출고로 합친 생산때 들어간 원재료: {0}".format(len(filtered_merged_run_release))  )

# filtered_merged_run_release



In [ ]:
pd.set_option('display.max_rows',None)
df_selected_841.loc[c1,'RMTL_CD'].str[:3].value_counts()

In [ ]:
aac = df_lwd333_lqlttr147_uniqueRM['RMTL_CD'].str[:3]=='AAC'
df_lwd333_lqlttr147_uniqueRM.loc[aac,'RMTL_CD'].value_counts().sort_index()

In [ ]:
aac_2 = df_selected_841['RMTL_CD'].str[:3]=='AAC'
df_selected_841.loc[c1 & aac_2,'RMTL_CD'].value_counts().sort_index()

In [ ]:
pd.set_option('display.max_rows',30)
filtered_df = merged_run_release[merged_run_release['SCAN_DTM'] <= merged_run_release['UDT_DTM']]
max_d_row = filtered_df.groupby(['MC_CD_x', 'LOT_NO', 'SPEC_CD', 'RMTL_CD'])['SCAN_DTM'].idxmax()
filtered_merged_run_release = filtered_df.loc[max_d_row]


print(  "정련출고로 합친 생산때 들어간 원재료: {0}".format(len(filtered_merged_run_release))  )

filtered_merged_run_release.head()


In [ ]:
print(len(df_selected_841[df_selected_841['MC_CD'].isna()]))
print(len(df_selected_841[~df_selected_841['MC_CD'].isna()]))

filtered_merged_run_release[ (filtered_merged_run_release['RMTL_CD']=='AAC421A')].sort_values(by='SCAN_DTM')

In [ ]:
df_lwd333_lqlttr147_uniqueRM [ (df_lwd333_lqlttr147_uniqueRM['RMTL_CD'].str[:3]=='AAC')]

In [ ]:
filtered_merged_run_release['RMTL_CD'].str[:3].value_counts()

In [ ]:
# merged_run_release

filtered_merged_run_release [ (filtered_merged_run_release['RMTL_CD'].str[:3]=='AAC')]

#### 설비가 있는 것 (투입이력이 정확히 남은경우) : 원부재, 공장, 설비 보고 합치기

In [ ]:
df_lwd333_lqlttr147_uniqueRM

In [ ]:
c2 = ~ df_selected_841['MC_CD'].isna()
merged_run_input = pd.merge(df_lwd333_lqlttr147_uniqueRM, df_selected_841[c2], on=['RMTL_CD', 'MC_CD', "PLT_CD"],how='inner')


# 투입 일자가 정련 시작보다 직전인 것 중에 가장 큰 값만 남기기
filtered_df_2 = merged_run_release[merged_run_release['SCAN_DTM'] <= merged_run_release['UDT_DTM']]

max_d_row_2 = filtered_df_2.groupby(['MC_CD_x', 'LOT_NO', 'SPEC_CD', 'RMTL_CD'])['SCAN_DTM'].idxmax()

filtered_merged_run_input = filtered_df_2.loc[max_d_row_2]

print(  "원부재투입LOT로 합친 생산때 들어간 원재료: {0}".format(len(filtered_merged_run_input))  )



filtered_merged_run_input

In [ ]:
# 최종 합치기
# 생산LOT 에 들어간 원료 연결한 최종본


raw_mixing_fin = pd.concat([filtered_merged_run_release, filtered_merged_run_input], axis=0).drop_duplicates()

del raw_mixing_fin['MC_CD_y']
raw_mixing_fin = raw_mixing_fin.rename(columns={'MC_CD_x': 'MC_CD'})
raw_mixing_fin.sort_values('LOT_NO')


# C1, C2 등 스텝 떼어내고 뒤에 컴파운드 스펙만 있는 행 만들기
raw_mixing_fin['SHORT_SPEC_CD'] = raw_mixing_fin['SPEC_CD'].str[2:]


# 폴더 이름 : 데이터 조회 기간
folder_name = 'Result_save/{0}_{1}'.format(ref_date1, ref_date2)

# 폴더가 존재하지 않는 경우 생성
if not os.path.exists(folder_name):
    os.mkdir(folder_name)


raw_mixing_fin.to_csv("Result_save/{0}_{1}/raw_mixing_fin {0}-{1}.csv".format (ref_date1, ref_date2))
raw_mixing_fin

In [ ]:
raw_mixing_fin [ (raw_mixing_fin['RMTL_CD'].str[:3]=='AAD')]

In [ ]:
set (df_lwd333_lqlttr147  [df_lwd333_lqlttr147['SPEC_CD'].str[2:]=='B87S053']['RMTL_CD'])

In [ ]:
raw_mixing_fin[raw_mixing_fin['RMTL_CD'].str[:3]=='AAC']

In [ ]:
set (raw_mixing_fin  [raw_mixing_fin['SPEC_CD'].str[2:]=='B87S053']['RMTL_CD'])

## 동일 CQ LOT에 들어가는 것 다 합치기

In [ ]:
df_lwd333_lqlttr147_uniqueRM[df_lwd333_lqlttr147_uniqueRM['LOT_NO']=='202301010003']

In [ ]:
c1 = df_lwd333_lqlttr147_uniqueRM['LOT_NO']=='202301020003'
c2 = df_lwd333_lqlttr147_uniqueRM['SPEC_CD']=='CQP90S043'
df_lwd333_lqlttr147_uniqueRM[c1&c2]

In [ ]:
# 모든 내용이 비어있는 열 삭제
df_trxx_trace = df_trxx_trace.dropna(axis=1, how='all')

In [ ]:
pd.set_option('mode.chained_assignment',  None) # 경고 off

# # 컬럼명 조정https://www.youtube.com/
# raw_mixing_fin = raw_mixing_fin.rename(columns={'LOT_NO':'MIXING_NO'})

# # 컬럼명 조정
# raw_mixing_fin = raw_mixing_fin.rename(columns={'LOT_NO':'MixingRunNO'})


df_trxx_trace['SHORT_SPEC_CD'] = df_trxx_trace['SPEC_CD'].str[2:]

# CQ 컬럼으로만 돌리기
df_trxx_trace_filtered = df_trxx_trace[ df_trxx_trace['SPEC_CD'].str[:2] == 'CQ']


trace_input_mat = []
# raw_mixing_fin2.set_index('Short Compd', inplace=True)


# trace 데이터 한 줄씩 돌아가면서 한 CQ 컴파운드+믹싱런에 들어간 C1, C2, C2 로뜨 리스트로 만들기
for i, row in df_trxx_trace_filtered.iterrows():
    shtcompd = row['SHORT_SPEC_CD']
    lotnum = row['LOT_NO']
    mccd = row['MC_CD']

    
  # trace CQ 믹싱런 한개로 모아지 C1, C2, C3단계의 로뜨넘버 모으기  ①
    step_lot = row[['LOT_NO', 'L1_LOT_NO', 'L2_LOT_NO', 'L3_LOT_NO', 'L4_LOT_NO', 'L5_LOT_NO']].tolist()
    step_lot = np.array([str(x) for x in step_lot if x != None])

  # trace CQ 믹싱런 한개에 있는 C1, C2, C3단계의 믹싱기 모으기  ②
    mixer =  row[['MC_CD','L1_MC_CD', 'L2_MC_CD', 'L3_MC_CD', 'L4_MC_CD', 'L5_MC_CD']].tolist()
    mixer = np.array([str(x) for x in mixer if x != None])


  # 컴파운드 명이 같고, 믹싱런과 설비가 위에  ①, ② 리스트에 포함된다면, 해당 행 맨 오른쪽에 trace CQ 믹싱런, 설비 적기
  # (그 행의 원료가 최종적으로 들어가는 CQ의 믹싱런넘버)
    c1 = raw_mixing_fin['SHORT_SPEC_CD'] == shtcompd
    c2 = raw_mixing_fin['LOT_NO'].isin(step_lot)  
    c3 = raw_mixing_fin['MC_CD'].isin(mixer) 
    condition = c1&c2&c3

    input_mat = raw_mixing_fin[condition]

    input_mat['FIN_LOT_NO']=len(input_mat) * [lotnum]
    input_mat['FIN_MC_CD']=len(input_mat) * [mccd]



    if not input_mat.empty:   trace_input_mat.append(input_mat)

trace_raw_runsum = pd.concat(trace_input_mat, ignore_index=True).drop_duplicates()
trace_raw_runsum  # 이게 CQ단계에 투입된 모든 원료 조회하는 최종 csv 형태임.


In [ ]:
pd.set_option('display.max_rows', 30)
# pd.reset_option('display.max_rows')

c1 =  trace_raw_runsum['MC_CD']=='A1003'
c2 = trace_raw_runsum['SPEC_CD']=='C1Y94S008'
# c3 = trace_raw_runsum['RMTL_CD']=='AAA101A'
c4 = trace_raw_runsum['LOT_NO']=='202301050001'

trace_raw_runsum [ c1&c2&c4].sort_values(by = 'BTCH_NO')

In [ ]:
# 믹싱런 하나당 한 배치에 들어간 원료들 조회한거랑 같음

trace_raw_runsum.to_csv("Result_save/{0}_{1}/trace_raw_runsum {0}-{1}.csv".format (ref_date1, ref_date2))


In [ ]:
pd.set_option('display.max_rows', 30)
trace_raw_runsum [ ( trace_raw_runsum['SPEC_CD']=='C1Y94S008' ) & ( trace_raw_runsum['RMTL_CD']=='AAE515A' )   ]

## 컴파운드 검사결과 합치기

#### ======= rpa랑 batch 검사 연결 ========

In [ ]:
# rpa검사
df_lqe060_lqlttr268 = df_lqe060_lqlttr268.dropna(axis = 1, how = 'all')

In [ ]:
c1 = df_lqe060_lqlttr268['SPEC_CD'] == 'CQP56S009'
c2 = df_lqe060_lqlttr268['MXG_PRDT_DATE'] == '20230220'

df_lqe060_lqlttr268[c1&c2].sort_values(by = 'BTCH_NO')

In [ ]:
# batch검사
df_lqe431_lqlttr171 = df_lqe431_lqlttr171.dropna (axis = 1, how = 'all')

In [ ]:
df_rpa_batch = pd.merge( df_lqe431_lqlttr171, df_lqe060_lqlttr268 , on=['PLT_CD', 'LOT_NO', 'SPEC_CD', 'MXG_MC_CD', 'BTCH_NO'], how='left')

non_nan_rows = df_rpa_batch[df_rpa_batch['G_50'].notnull()]

print('매칭된 rpa개수:' ,len(non_nan_rows))
df_rpa_batch.rename(columns = {'MXG_PRDT_DATE_x' : 'MXG_PRDT_DATE'}, inplace = True)

df_rpa_batch

In [ ]:
# # 통합
# search_SPEC_CD = 'CQU33S098'
# search_MXG_PRDT_DATE = '20230306'
# search_MXG_MC_CD = 'A1015'

# c1 = df_rpa_batch['SPEC_CD']== search_SPEC_CD
# c2 = df_rpa_batch['MXG_PRDT_DATE']==search_MXG_PRDT_DATE
# c3 = df_rpa_batch['MXG_MC_CD'] == search_MXG_MC_CD
# c4 = df_rpa_batch['G_50'].notnull()

# # rpa
# d1 = df_lqe060_lqlttr268['SPEC_CD']== search_SPEC_CD
# d2 = df_lqe060_lqlttr268['MXG_PRDT_DATE']== search_MXG_PRDT_DATE
# d3 = df_lqe060_lqlttr268['MXG_MC_CD'] ==search_MXG_MC_CD

# # batch
# e1 = df_lqe431_lqlttr171['SPEC_CD']==search_SPEC_CD
# e2 = df_lqe431_lqlttr171['MXG_PRDT_DATE']==search_MXG_PRDT_DATE
# e3 = df_lqe431_lqlttr171['MXG_MC_CD'] == search_MXG_MC_CD

# pd.set_option('display.max_rows', 30)

In [ ]:
# df_rpa_batch[c1&c2&c3&c4].sort_values(by=['MXG_LOT_NO','BTCH_NO'])

In [ ]:
# df_lqe431_lqlttr171[e1&e2&e3].sort_values(by=['MXG_LOT_NO','BTCH_NO'])

In [ ]:
# df_lqe060_lqlttr268[d1&d2&d3].sort_values(by=['LOT_NO','BTCH_NO'])

In [ ]:
df_rpa_batch.to_csv("Result_save/{0}_{1}/df_rpa_batch {0}-{1}.csv".format(ref_date1, ref_date2), encoding="cp949")

#### ==========고무물성(BIS,MV, Rheo)결과도 연결하기===============

In [ ]:
df_lqe461_lqlttr177 = df_lqe461_lqlttr177.dropna(axis = 1, how = 'all')

In [ ]:
df_rpa_batch_property = pd.merge(df_rpa_batch, df_lqe461_lqlttr177, how='left', on=['MXG_PRDT_DATE', 'PLT_CD', 'SPEC_CD', 'MXG_MC_CD', 'BTCH_NO'])
df_rpa_batch_property['SHORT_SPEC_CD'] = df_rpa_batch_property['SPEC_CD'].str[2:]


df_rpa_batch_property

In [ ]:
non_nan_rows = df_rpa_batch_property[df_rpa_batch_property['IF_FG'].notnull()]

print('고무물성 채워진 개수:',len( non_nan_rows))

In [ ]:
from tabulate import tabulate

# 각 컬럼별 비어있는 행 수
empty_row_counts = df_rpa_batch_property.isnull().sum()

# 각 컬럼별 비어있는 비율
empty_row_ratios = (empty_row_counts / len(df_rpa_batch_property))

# 각 컬럼별 안비어있는 행 수
non_empty_row_counts = df_rpa_batch_property.notnull().sum()


# # 모든 행이 비어있는 컬럼 확인 및 리스트 생성
# all_empty_columns = empty_row_ratios[empty_row_ratios == 1].index.tolist()

# 결과 출력
data_sum_result = pd.DataFrame({
    '컬럼명': empty_row_counts.index,
    '채워진 수': non_empty_row_counts.values,
    'null 수': empty_row_counts.values,
    'null 비율': empty_row_ratios.values
})

# 모든 행이 비어있는 컬럼 리스트 추가
data_sum_result['ALL_NULL'] = data_sum_result['컬럼명'].isin(all_empty_columns)


table = tabulate(data_sum_result, headers='keys', tablefmt='pretty', showindex=False)
print(table)

In [ ]:
df_rpa_batch_property = df_rpa_batch_property.dropna (axis = 1, how='all')
df_rpa_batch_property = df_rpa_batch_property.drop_duplicates()


df_rpa_batch_property.to_csv("Result_save/{0}_{1}/df_rpa_batch_property {0}-{1}.csv".format(ref_date1, ref_date2))

## ====== 모든 데이터 다 잇기 ========


In [ ]:
df_rpa_batch_property

In [ ]:
trace_raw_runsum

In [ ]:
# 모든 배치, RPA 검사, 고무물성  -  업체 / LOT 연결하는 작업
df_final = pd.merge( df_rpa_batch_property, trace_raw_runsum , left_on=['PLT_CD','MXG_MC_CD', 'MXG_LOT_NO', 'SPEC_CD'], right_on=['PLT_CD', 'FIN_MC_CD','FIN_LOT_NO', 'SPEC_CD'], how='left')

# df_final = df_final.drop(['PLT_CD', '최종LOT_NO', 'SHORT_SPEC_CD', 'MC_CD'], axis = 1)
df_final = df_final.drop_duplicates()


df_final.rename(columns = {'LOT_NO_y':'PRE_LOT_NO'}, inplace = True)
df_final.rename(columns = {'SPEC_CD_y':'PRE_SPEC_CD'}, inplace = True)
df_final.rename(columns = {'MC_CD':'PRE_MC_CD'}, inplace = True)


In [ ]:
df_final

In [ ]:
c1 = df_final['MXG_LOT_NO'] == '202302120003'
c2 = df_final['SPEC_CD'] == 'CQB87S053'
# c3 = df_final['RMTL_CD'].str[:3] == 'AAS'


set (df_final[c1&c2].sort_values(by='BTCH_NO_x')['RMTL_CD'].to_numpy())

In [ ]:
# 각 컬럼별 비어있는 행 수
empty_row_counts = df_final.isnull().sum()

# 각 컬럼별 비어있는 비율
empty_row_ratios = (empty_row_counts / len(df_final))

# 각 컬럼별 안비어있는 행 수
non_empty_row_counts = df_final.notnull().sum()


# 모든 행이 비어있는 컬럼 확인 및 리스트 생성
all_empty_columns = empty_row_ratios[empty_row_ratios == 1].index.tolist()

# 결과 출력
data_result = pd.DataFrame({
    '컬럼명': empty_row_counts.index,
    '채워진 수': non_empty_row_counts.values,
    'null 수': empty_row_counts.values,
    'null 비율': empty_row_ratios.values
})


# 모든 행이 비어있는 컬럼 리스트 삭제
data_result['ALL_NULL'] = data_result['컬럼명'].isin(all_empty_columns)
df_final = df_final.drop(columns=all_empty_columns)

# 정보 중복된 컬럼 삭제
duplicated_columns = ['FIN_LOT_NO', 'FIN_MC_CD',  'LOT_NO_x']
df_final = df_final.drop(columns=duplicated_columns)

# 원료정보 + CQ 로뜨정보 없는 행 삭제
df_final = df.final.dropna(subset=['CMP_NM'])
df_final = df_final.dropna(subset=['FIN_LOT_NO'])

table = tabulate(data_result, headers='keys', tablefmt='pretty', showindex=False)
print(table)
df_final

In [ ]:
c1 = df_final['FIN_LOT_NO']=='202302060011'
c2 = df_final['FIN_MC_CD']=='A1015'
c3 = df_final['RMTL_CD']=='AAQ213A'

df_final[c1&c2&c3]

In [ ]:
df_final.to_csv("Result_save/{0}_{1}/df_final {0}-{1}_2.csv".format(ref_date1, ref_date2), encoding = 'utf-8-sig')